In [33]:
import pandas as pd
import numpy as np
import torch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [34]:
torch.manual_seed(42)

In [35]:
df = pd.read_csv('fashion-mnist_train.csv')


In [36]:
print(torch.cuda.is_available())

True


In [8]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df.head(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df.head(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
x = df.iloc[0].tolist()[1:]  # features (all columns except first)
y = df.iloc[0].tolist()[0]   # label (first column)


In [40]:
y=df['label'].values

In [41]:
X=df.iloc[:,1:].values

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2, random_state=42)


In [43]:
X_train=X_train/255
X_test=X_test/255

In [44]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features= torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index) :
        return self.features[index] , self.labels[index]

In [45]:
train_dataset = CustomDataset(X_train,y_train)


In [46]:
len(train_dataset)

48000

In [47]:
test_dataset = CustomDataset(X_test,y_test)


In [48]:
# Step 2: Load FAISS index and connect it with HuggingFace LLM for retrieval-augmented answering


In [49]:
len(test_dataset)

12000

In [50]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [23]:
# from turtle import forward


# class MySimpleNN(nn.Module):
#     def __init__(self,num_features):
#         super().__init__()
#         self.model = nn.Sequential(
#             nn.Linear(num_features,128),
#             nn.BatchNorm1d(128),
#             nn.ReLU(),nn.Dropout(p=0.3),
#             nn.Linear(128,64),
#             nn.BatchNorm1d(64),
#             nn.ReLU(),nn.Dropout(p=0.3),
#             nn.Linear(64,10)
#             )
#     def forward(self,X):
#         return self.model(X)





In [69]:
class MySimpleNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.features = nn.Sequential(  # renamed from feature -> features
            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7*7*64, 128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )

    def forward(self, X):
        x = self.features(X)
        out = self.classifier(x)
        return out


In [59]:
epochs= 100
learning_rate = 0.1


In [60]:
type(X_train.shape[1])

int

In [61]:
import torch.optim as optim


In [62]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [70]:
model = MySimpleNN(1)
model=model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = learning_rate,weight_decay=1e-4)

In [ ]:
for epoch in range(epochs):
    total_epochs_loss = 0
    for batch_features, batch_labels in train_loader:
      batch_features = batch_features.to(device)
      batch_labels = batch_labels.to(device)
      outputs = model(batch_features)
      loss = criterion(outputs,batch_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_epochs_loss=total_epochs_loss+loss.item()
    avg_loss=total_epochs_loss/len(train_loader)
    print("loss: ",avg_loss,' epoch: ',epoch)



loss:  0.5274889644682408  epoch:  0
loss:  0.35965157675991455  epoch:  1
loss:  0.3157517090936502  epoch:  2
loss:  0.2811413937999556  epoch:  3
loss:  0.25291702477633954  epoch:  4
loss:  0.2284687374358376  epoch:  5
loss:  0.2112915579744925  epoch:  6
loss:  0.1958269445821643  epoch:  7
loss:  0.181699209679539  epoch:  8
loss:  0.1712476492424806  epoch:  9
loss:  0.1595914064142853  epoch:  10
loss:  0.15145625180890784  epoch:  11
loss:  0.14692446172175308  epoch:  12
loss:  0.13753189859135698  epoch:  13
loss:  0.13392020983388647  epoch:  14
loss:  0.1251914336400417  epoch:  15


In [30]:
model.eval()

MySimpleNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [31]:




total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
      batch_features = batch_features.to(device)
      batch_labels = batch_labels.to(device)
      outputs = model(batch_features)
      _, predicted= torch.max(outputs,1)
      total= total +  batch_labels.shape[0]
      correct = correct + (predicted == batch_labels).sum().item()
    print(correct/total)



0.88325


In [32]:




total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in train_loader:
      batch_features = batch_features.to(device)
      batch_labels = batch_labels.to(device)
      outputs = model(batch_features)
      _, predicted= torch.max(outputs,1)
      total= total +  batch_labels.shape[0]
      correct = correct + (predicted == batch_labels).sum().item()
    print(correct/total)



0.9325416666666667
